# 🧪 Steering Playground: Esperimenti Avanzati (Versione 2.0)

Benvenuto nel playground aggiornato! Ora include:
1.  **Vettori Robusti**: Possiamo usare LISTE di frasi per pulire il rumore.
2.  **Layer Sweep**: Un tool per trovare il layer perfetto.
3.  **Normalizzazione Automatica**: Niente più farfugliamenti.

In [10]:
%load_ext autoreload
%autoreload 2

import torch
from steering_lib import ModelloSteerable
import prompts

# Carichiamo il modello
modello = ModelloSteerable(model_name="gpt2", device="cuda")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Caricamento modello gpt2 su cuda...


## 🛠️ Funzione Helper Aggiornata
Ora supporta anche il Layer Sweep!

In [11]:
def esegui_esperimento(titolo, prompt_pos, prompt_neg, prompt_test, layer=6, forza=5.0):
    print(f"\n{'='*20} {titolo} {'='*20}")
    # Non stampiamo tutto se sono liste lunghe, solo il primo elemento per info
    p_pos_display = prompt_pos[0] if isinstance(prompt_pos, list) else prompt_pos
    p_neg_display = prompt_neg[0] if isinstance(prompt_neg, list) else prompt_neg
    
    print(f"🟢 Concetto Positivo: '{p_pos_display}...' (o lista)")
    print(f"🔴 Concetto Negativo: '{p_neg_display}...' (o lista)")
    print(f"📝 Prompt di Test:    '{prompt_test}'")
    
    # 1. Calcolo del vettore (ora supporta liste!)
    vettore = modello.estrai_vettore(prompt_pos, prompt_neg, layer)
    
    print(f"\n--- 1. Baseline ---")
    print(modello.genera(prompt_test, moltiplicatore=0))
    
    print(f"\n--- 2. Steering Attivato (Layer {layer}, Forza {forza}) ---")
    print(modello.genera(prompt_test, vettore_steering=vettore, layer_idx=layer, moltiplicatore=forza))
    
    print(f"\n--- 3. Steering Opposto (Layer {layer}, Forza -{forza}) ---")
    print(modello.genera(prompt_test, vettore_steering=vettore, layer_idx=layer, moltiplicatore=-forza))

## 🚀 Novità: Layer Sweep
Proviamo lo stesso vettore su TUTTI i layer per vedere dove funziona meglio.

In [12]:
def layer_sweep(prompt_pos, prompt_neg, prompt_test, layers=range(12), forza=5.0):
    print(f"\n🔍 Eseguendo Layer Sweep...")
    for layer in layers:
        # Calcoliamo il vettore specifico per questo layer
        vettore = modello.estrai_vettore(prompt_pos, prompt_neg, layer)
        # Generiamo
        output = modello.genera(prompt_test, vettore_steering=vettore, layer_idx=layer, moltiplicatore=forza, max_new_tokens=15)
        # Stampiamo solo la parte generata (pulizia)
        generato = output[len(prompt_test):].strip().replace('\n', ' ')
        print(f"[Layer {layer:2d}]: ... {generato}")

## Esperimento 1: Fantasy vs Sci-Fi (ROBUSTO)
Usiamo liste di frasi per definire meglio il genere.

In [24]:
'''
fantasy_prompts = [
    "A magical sword glowing with ancient runes.",
    "The old wizard cast a spell of fire.",
    "A dragon flying over the castle.",
    "The elven kingdom in the forest."
]

scifi_prompts = [
    "A laser blaster with advanced targeting systems.",
    "The spaceship engaged its warp drive.",
    "An artificial intelligence controlling the city.",
    "Cybernetic implants and neon lights."
]
'''

esegui_esperimento(
    titolo="Genere: Fantasy vs Sci-Fi (Robusto)",
    prompt_pos=prompts.FANTASY_PROMPTS,  # 
    prompt_neg=prompts.SCIFI_PROMPTS,   # 
    prompt_test="In the box, I found a",
    layer=8,
    forza=40.0
)


==================== Genere: Fantasy vs Sci-Fi (Robusto) ====================
🟢 Concetto Positivo: 'A magical sword glowing with ancient runes....' (o lista)
🔴 Concetto Negativo: 'A laser blaster with advanced targeting systems....' (o lista)
📝 Prompt di Test:    'In the box, I found a'

--- 1. Baseline ---
In the box, I found a set of a variety of accessories with a small space. It was hard to put my finger on what exactly these included.


We had some other sets on hand, and it seemed as though some of them were underpowered. (I had a

--- 2. Steering Attivato (Layer 8, Forza 40.0) ---
In the box, I found a very nice long, deep wooden sword. It's not particularly sharp, but is extremely large. It's carved from stone, and I can see the hilt coming out. I'll have to eat it! Aaa-kay-aa-ay

--- 3. Steering Opposto (Layer 8, Forza -40.0) ---
In the box, I found a few basic features, including:


USB-C for USB devices to connect to and communicate with devices

USB-C for USB communication

### Analisi dei Layer per Fantasy
Vediamo quale layer è il migliore per questo concetto.

In [21]:
layer_sweep(prompts.FANTASY_PROMPTS, prompts.SCIFI_PROMPTS, "In the box, I found a", forza=10.0)


🔍 Eseguendo Layer Sweep...
[Layer  0]: ... list of seven possible outcomes for a specific outcome:  "B-
[Layer  1]: ... small bag of books on the floor of the room, the books I'd
[Layer  2]: ... sample of a white-painted gold chain around the center.  Using
[Layer  3]: ... handful of pages with more information for each title, but I also searched for
[Layer  4]: ... piece of paper. A few minutes later I found a paper which I had
[Layer  5]: ... 2nd copy of Star Destroyer and I used it to build up some pretty
[Layer  6]: ... bunch of other new stuff too, but unfortunately it didn't give me many
[Layer  7]: ... 3-panel version. One of the corners at the front has a white
[Layer  8]: ... small set of three plastic cups. I thought, "Now I might just
[Layer  9]: ... nice white case that came with a very nice wooden handle.   This
[Layer 10]: ... sample of 12 ounces of corn syrup and two tablespoons of sugar, and placed
[Layer 11]: ... 3" square blade with a 10/9" blade and a 1/


## Esperimento 2: Stile Formale vs Street (ROBUSTO)
Usiamo più esempi per catturare il "tono".

In [19]:
'''
formal_prompts = [
    "I would like to formally request assistance.",
    "Please consider this matter with urgency.",
    "The data suggests a significant correlation."
]

street_prompts = [
    "Yo bro, help me out.",
    "Gimme that thing right now.",
    "Whatever man, it's all good."
]
'''
esegui_esperimento(
    titolo="Stile Robusto",
    prompt_pos=prompts.FORMAL_PROMPTS,
    prompt_neg=prompts.STREET_PROMPTS,
    prompt_test="Hey, I need",
    layer=7, 
    forza=40.0
)


==================== Stile Robusto ====================
🟢 Concetto Positivo: 'I would like to formally request assistance....' (o lista)
🔴 Concetto Negativo: 'Yo bro, help me out....' (o lista)
📝 Prompt di Test:    'Hey, I need'

--- 1. Baseline ---
Hey, I need to go back to your house. I can pick you up easily. Don't worry. Come with me. I can go back to your house and clean my house. If you don't mind."

"Okay."

"Then I

--- 2. Steering Attivato (Layer 7, Forza 40.0) ---
Hey, I need some assistance with the information that you have reported on our site and the information that has been provided under this Policy.

The information referred to in this Policy is that of our employees, which will be provided to all of our employees, the extent

--- 3. Steering Opposto (Layer 7, Forza -40.0) ---
Hey, I need a bed!" "I was talking on my cellphone before you came out!" "Just go, I'll be okay!" "Oh... I'm sorry, okay!" ...Huh oh Oh. What's that? Don't go over there, don't


## Esperimento 3: Love vs Hate (Sanity Check)
Verifica funzionale su emozioni primarie.
Il modello gpt2 essendo un modello piccolo dovrebbe essere in grado di capire le emozioni primarie più facilmente rispetto a concetti più complessi. Questo esperimento dovrebbe performare meglio degli altri.

In [17]:
'''
love_prompts = [
    "Love is the most powerful force.",
    "I feel so happy and joyful.",
    "My heart is full of affection.",
    "This is wonderful and delightful."
]

hate_prompts = [
    "Hate is destroying everything.",
    "I feel angry and disgusted.",
    "My heart is cold and bitter.",
    "This is terrible and miserable."
]
'''
esegui_esperimento(
    titolo="Sanity Check: Amore vs Odio",
    prompt_pos=prompts.LOVE_PROMPTS,
    prompt_neg=prompts.HATE_PROMPTS,
    prompt_test="I think this movie is",
    layer=7,
    forza=40.0
)


==================== Sanity Check: Amore vs Odio ====================
🟢 Concetto Positivo: 'Love is the most powerful force....' (o lista)
🔴 Concetto Negativo: 'Hate is destroying everything....' (o lista)
📝 Prompt di Test:    'I think this movie is'

--- 1. Baseline ---
I think this movie is a good thing and makes me want to watch more movies," he says. "The story is a complete rewrite. That's why my interest in seeing those movies went up."

Even when he hadn't expected an adaptation to be this big,

--- 2. Steering Attivato (Layer 7, Forza 40.0) ---
I think this movie is the most diverse I've ever had you look at anything in the industry and you have it so easy. Because the girls are always doing well."

--- 3. Steering Opposto (Layer 7, Forza -40.0) ---
I think this movie is completely fucked up with the fact that there's a whole world of people who are asking. How exactly do they think you and your government would really not have the ability to say, 'Well, how do you understand 